In [45]:
###Rather than reinitializing maybe update keys
class keys:
    def __init__(self, event= None):
        self.string = "Pressed:"
        if event.key == pygame.K_UP:
            self.UP = True
            self.string += "UP"
        else:
            self.UP = False
        if event.key == pygame.K_DOWN:
            self.DOWN = True
            self.string += "DOWN"
        else:
            self.DOWN = False
        if event.key == pygame.K_RIGHT:
            self.RIGHT = True
            self.string += "RIGHT"
        else:
            self.RIGHT = False    
        if event.key == pygame.K_LEFT:
            self.LEFT = True
            self.string += "LEFT"
        else:
            self.LEFT = False 
 
    def __str__(self):
        return self.string
        
        
def xy_position_to_pixels(Rectangle, window_size):
    """Takes:In-game coordinate rectangle
    Converts in game coordinates to pixels. Also, note that the in-game position refers to the LOWER RIGHT hand
    corner of the sprite. 
    Returns: rectangle in pixel coordinates"""
    x = Rectangle[0]
    y = Rectangle[1]
    width = Rectangle[2]
    height = Rectangle[3]
    pixel_x = int(float(x/300)*window_size[0])
    pixel_y = window_size[1] - int(float(y/200)*window_size[1])
    pixel_width = int(float(width/300)*window_size[0])
    pixel_height = int(float(height/200)*window_size[1])
    pixel_x -= pixel_width
    pixel_y -= pixel_height
    return (pixel_x, pixel_y, pixel_width, pixel_height)



class position:
    def __init__(self, current_x = 0, current_y = 0, current_xs = 0, current_ys = 0, floor = 0, button=None):
        self.current_x = current_x
        self.current_y = current_y
        self.current_xs = current_xs
        self.current_ys = current_ys
        self.button = button
        self.floor = floor
        self.jump = False

    def __str__(self):
        return str(self.current_x)+" "+str(self.current_y)+" "+str(self.current_xs)+" "+str(self.current_ys)
    
    def give_next_position(self): #fix button 

        #print("The give next position function has run.")
        #print("Button:", button)
        if self.current_y == self.floor:
            self.current_ys = 0
            self.jump = False
        self.current_xs = 0
        
        if self.button != None: #FOUND PROBLEM ONLY UPDATES WITH BUTTON PRESS
            print("AH I SENSE MOVEMENT!")
            
            if self.current_y == self.floor:
                if self.jump == False and self.button.UP:
                    print("this should show up once")
                    self.current_ys = 10
                    self.jump = True
                    
            if self.current_y > self.floor:
                self.jump = True
                self.current_ys -= 1
                
            if self.button.RIGHT:
                self.current_xs = 1
            elif self.button.LEFT:
                self.current_xs = -1
        
            
        self.current_x += self.current_xs
        self.current_y += self.current_ys
        if self.current_y < self.floor:
            self.current_y = self.floor
        

In [46]:
import pygame


pygame.init()
window_size = (pygame.display.Info().current_w -100, pygame.display.Info().current_h - 100)
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("Egg World")
done = False
clock = pygame.time.Clock()
BLUE = (0,0,255)

current_position = position()  

x = 0
y = 0
#timer_event = pygame.USEREVENT + 1
#pygame.time.set_timer(timer_event, 250) #makes it so that your not waiting for events to run
while not done:
    clock.tick(50)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                done = True
            print("KEY DOWN/UP!")
            current_position.button = keys(event)
            print(current_position.button)
        if event.type == pygame.KEYUP:
            current_position.button = None
            
    print("Outside loop sees ", current_position.button)
    current_position.give_next_position()
    
    screen.fill((0,0,50))
    Rectangle = xy_position_to_pixels((current_position.current_x, current_position.current_y,10,10), window_size)
    pygame.draw.rect(screen, BLUE, Rectangle)
    pygame.display.flip()
        
        
pygame.quit()



Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
Outside loop sees  None
KEY DOWN/UP!
Pressed:RIGHT
Outside loop sees  Pressed:RIGHT
AH I SENSE MOVEMENT!
Outside loop sees  Pressed:RIGHT
AH I SENSE MOVEMENT!
Outside loop sees  Pressed:RIGHT
AH I SENSE MOVEMENT!
Outside loop sees  